In [97]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import yfinance as yf
import margo_loader
import pecnet_model as pcm
import finance_db as fdb


In [98]:
all_data=pd.DataFrame()
ticker_list=["AAPL","MSFT"]

def get_data(tickerlist):
    return yf.download(tickerlist,start = "2000-01-01",end="2023-08-21")[["Adj Close"]] #to include the end day, add 1 day

In [99]:
def get_adjcloseprice_and_plot():
    
    df_stock_prices=get_data(ticker_list)["Adj Close"]
    print(df_stock_prices)
    
    plt.figure(figsize=(15,5))
    plt.ylabel('ClosePrice ($)', fontsize=16)
    plt.title("StockPrices", fontsize=16)

    for ticker in ticker_list:
        plt.plot(df_stock_prices.index,df_stock_prices[ticker],label=ticker)

    plt.legend()
    plt.show()

#get_adjcloseprice_and_plot()

In [100]:
def get_predictions(stock_prices):

    prediction_dict={}


    for ticker in ticker_list:
        if stock_prices.empty:
            continue
        
        input_data=np.array(stock_prices["Adj Close"][ticker])
        input_dates=np.array(stock_prices.index.astype(str, copy = False))
        

        nanmask=np.isnan(input_data)

        input_data=input_data[~nanmask]
        input_dates=input_dates[~nanmask]

        predictions = pcm.train_and_test(input_data)
        test_index=len(input_data)+1-len(predictions) #last prediction value is tomorrow's prediction
        real_values=input_data[test_index:]
        error_values=predictions[:-1]-real_values

        predictions = [ round(elem, 3) for elem in predictions ]
        real_values = [ round(elem, 3) for elem in real_values ]
        error_values = [ round(elem, 3) for elem in error_values ]
        date_values=input_dates[test_index:]

        #for the tomorrow prediction

        real_values.append(None)
        error_values.append(None)
        #prediction is already ok
        #add 1 more date to date_values
        tomorrow=str(pd.to_datetime(input_dates[-1])+pd.offsets.BDay(1))
        date_values=np.append(date_values,tomorrow)

        assert len(date_values) == len(real_values)
        
        prediction_dict[ticker]=pd.DataFrame({"date":date_values,"adj_close_price":real_values,"predicted_price":predictions,"error":error_values})
    
    return prediction_dict



In [101]:
def get_all_data_columns():
    data = {}
    for ticker in ticker_list:
        predictions = fdb.get_predictions_from_db(ticker)
        data[ticker] = pd.DataFrame(predictions, columns=['id', 'date', f'adj_close_price', f'predicted_price', f'error']).set_index('date').drop('id',axis=1)
        
    return data


def get_predictions_and_save():
    global all_data
    
    stock_prices=get_data(ticker_list)
    result_dict=get_predictions(stock_prices)
    
    for ticker in result_dict:
        result=fdb.add_predictions_to_db(ticker,result_dict[ticker])

        yesterday=str(pd.to_datetime(result_dict[ticker]["date"].iloc[-1])-pd.offsets.BDay(1))#last date is tomorrow so yesterday is today in fact.
        
        if fdb.is_date_available(yesterday,ticker):
            gt_yesterday=round(stock_prices["Adj Close"][ticker].iloc[-1],3)
            err_yesterday=round(fdb.get_predictions_from_db(ticker,dt.strptime(yesterday,"%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d"))[0][3]-gt_yesterday,3) # 0 is first elem in the tuple list.3 is the index of the prediction column
            
            fdb.update_previous_ground_truth(yesterday,ticker,gt_yesterday,err_yesterday)
    
    all_data = get_all_data_columns()


def get_prediction_df():
    table = pd.DataFrame(columns=['date', 'adj_close_price', 'predicted_price', 'action'])
    for ticker in ticker_list:
        df = all_data[ticker].iloc[-2:][['adj_close_price', 'predicted_price']]
        action = 'SELL' if df.iloc[0].adj_close_price > df.iloc[1].predicted_price else 'BUY'
        table.loc[ticker] = df.iloc[0].name, df.iloc[0].adj_close_price, df.iloc[1].predicted_price, action

    return table


all_data = get_all_data_columns()
#get_predictions_and_save()

[*********************100%%**********************]  2 of 2 completed


c:\ProgramData\Anaconda3\envs\pecnetnasdaq\Lib\site-packages\keras\src\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 0s 22ms/step


c:\ProgramData\Anaconda3\envs\pecnetnasdaq\Lib\site-packages\keras\src\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 0s 24ms/step
